# Aims

* To identify what topics are shared in the documents
    * most similar
    * most dissimilar
* To plot the similarity matrix
* To have this as a template to feed in model and corpus

In [ ]:
from gensim import corpora, models, similarities
import gensim
import logging
import stop_words
import nltk
import string
import os
from nltk.stem import WordNetLemmatizer
import numpy as np
import numpy
import pickle
from operator import itemgetter
import pandas as pd

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
lda = gensim.models.ldamodel.LdaModel.load("data2lda10.lda")
# lda2 = gensim.models.ldamodel.LdaModel.load("data2lda200.lda")
# lda4 = gensim.models.ldamodel.LdaModel.load("data2lda400.lda")
with open("corp.cor") as cp: 
    corp = pickle.load(cp)
with open("sentances.sent") as sent: 
    sentances = pickle.load(sent)
dictionary = gensim.corpora.dictionary.Dictionary.load("Dicktionary")

In [ ]:
def findTopics(query):
    vec_bow = dictionary.doc2bow(query)
    vec_lsi = lda.get_document_topics(vec_bow) # convert the query to LSI space
#     sims = sim_index[vec_lsi]
    vec_lsi = [list(x) for x in vec_lsi]
#     print vec_lsi
#     vec_lsi = vec_lsi.sort(key=itemgetter[1])
    return(vec_lsi) # print sorted (document number, similarity score) 2-tuples
    

THis is a % similarity... however, I want to implement a jaccard similairty to get a legit methodology. 

Looks like you take a binary ... wait. No, I dont understand it. 

In [ ]:
topics_freqdist = []

for item in sentances: 
    try:
        ass_topic = findTopics(item)
        ass_topic.sort(key=itemgetter(1), reverse = True)
#         ass_topic = ass_topic[0][0]
        print ass_topic
#         print ass_topic
        topics_freqdist.append(ass_topic)
#         print ""
    except Exception, e: 
        print e
    

# print topics_freqdist

In [ ]:
sim_dict = {}
for doc in topics_freqdist: 
    for topic in doc: 
        if topic[0] not in sim_dict: 
            sim_dict[topic[0]] = {}
        for related_topic in doc:
            if related_topic[0] not in sim_dict[topic[0]]: 
                sim_dict[topic[0]][related_topic[0]] = related_topic[1]
            else:
                sim_dict[topic[0]][related_topic[0]] += related_topic[1]
                
for topic in sim_dict: 
    for related_topic in sim_dict[topic]: 
        sim_dict[topic][related_topic] = sim_dict[topic][related_topic]/float(len(topics_freqdist))
                
# for item in sim_dict:
#     print item, sim_dict[item]

In [ ]:
df = pd.DataFrame.from_dict(sim_dict).T.fillna(0)

print df

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(df, cmap = "hot", interpolation = "nearest")

plt.show()

In [ ]:
import networkx as nx
import graphviz

df_stack = df.stack().reset_index().rename(columns = {"level_0": "Node1", "level_1": "Node2", 0: "Similarity"})

mask = df_stack.Similarity < 0.04

column_name = "Similarity"

df_stack.loc[mask, column_name] = None

df_stack = df_stack.dropna()

# print df_stack

network = nx.from_pandas_dataframe(df_stack, "Node1", "Node2", ["Similarity"])

# colors=range(len(df_stack))

nx.draw_networkx(network)


plt.show()

In [ ]:
import networkx as nx
import graphviz

df_stack = df2.stack().reset_index().rename(columns = {"level_0": "Node1", "level_1": "Node2", 0: "Similarity"})

mask = df_stack.Similarity < 0.04

column_name = "Similarity"

df_stack.loc[mask, column_name] = None

df_stack = df_stack.dropna()

# print df_stack

network = nx.from_pandas_dataframe(df_stack, "Node1", "Node2", ["Similarity"])

# colors=range(len(df_stack))

nx.draw_networkx(network)


plt.show()


In [ ]:
topics_vis = lda.show_topics(20)

topics_vis = sorted(topics_vis, key = itemgetter(0))

# print topics_vis

print len(topics_vis)

In [ ]:
from sklearn.cluster import SpectralClustering

numpy_array = df.as_matrix()

num_clusters = 3

spectral = SpectralClustering(num_clusters).fit_predict(numpy_array)

for second_topic_num in range(num_clusters):
    print second_topic_num
    for second_cluster, topic in zip(spectral, topics_vis):
        if second_topic_num == second_cluster:
            print topic
    print " "
    print ""

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corp, dictionary)